# Face Emotion Recognition

### Importing the librariesNew Section

In [ ]:
import os
import cv2
import glob
import numpy as np
from time import sleep
from google.colab.patches import cv2_imshow


import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from keras.models import load_model
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Loading the input data

In [ ]:
num_classes = 7
img_rows,img_cols = 48,48
batch_size = 32

train_data_dir = '/content/drive/MyDrive/Colab Notebooks/Machine Learning/Project/Dataset/train'
validation_data_dir = '/content/drive/MyDrive/Colab Notebooks/Machine Learning/Project/Dataset/test'

### Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
					rescale=1./255,
					rotation_range=30,
					shear_range=0.3,
					zoom_range=0.3,
					width_shift_range=0.4,
					height_shift_range=0.4,
					horizontal_flip=True,
					fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
					train_data_dir,
					color_mode='grayscale',
					target_size=(img_rows,img_cols),
					batch_size=batch_size,
					class_mode='categorical',
					shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
							validation_data_dir,
							color_mode='grayscale',
							target_size=(img_rows,img_cols),
							batch_size=batch_size,
							class_mode='categorical',
							shuffle=True)

Found 28763 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


# CNN

In [ ]:
# Train the model

model = Sequential()

# Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-2 

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-4 

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-5

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-6

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-7

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 48, 48, 32)        320       
                                                                 
 activation_11 (Activation)  (None, 48, 48, 32)        0         
                                                                 
 batch_normalization_10 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                                
                                                                 
 conv2d_9 (Conv2D)           (None, 48, 48, 32)        9248      
                                                                 
 activation_12 (Activation)  (None, 48, 48, 32)        0         
                                                                 
 batch_normalization_11 (Bat  (None, 48, 48, 32)       128       
 chNormalization)                                     

### Train the model 

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Machine Learning/Project/Emotion_vgg.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])
epochs=25

history=model.fit_generator(
                train_generator,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=batch_size)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/25
357/357 [==============================] - ETA: 0s - loss: 1.8829 - accuracy: 0.3303
Epoch 00001: val_loss improved from inf to 1.88111, saving model to drive/My Drive/Face Emotion Recognition/Emotion_vgg.h5
357/357 [==============================] - 2987s 8s/step - loss: 1.8829 - accuracy: 0.3303 - val_loss: 1.8811 - val_accuracy: 0.2296 - lr: 0.0010
Epoch 2/25
357/357 [==============================] - ETA: 0s - loss: 1.4724 - accuracy: 0.4146
Epoch 00002: val_loss did not improve from 1.88111
357/357 [==============================] - 60s 169ms/step - loss: 1.4724 - accuracy: 0.4146 - val_loss: 1.9543 - val_accuracy: 0.2365 - lr: 0.0010
Epoch 3/25
357/357 [==============================] - ETA: 0s - loss: 1.3733 - accuracy: 0.4630
Epoch 00003: val_loss did not improve from 1.88111
357/357 [==============================] - 61s 171ms/step - loss: 1.3733 - accuracy: 0.4630 - val_loss: 1.9647 - val_accuracy: 0.2416 - lr: 0.0010
Epoch 4/25
357/357 [===========================

Train the model by removing the Early stopping point but by reducing the learning rate on plateau.

In [ ]:
checkpoint = ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/Machine Learning/Project/Emotion_vgg2.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=4,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [checkpoint,reduce_lr]

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

epochs=25

history=model.fit_generator(
                train_generator,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=batch_size)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/25
357/357 [==============================] - ETA: 0s - loss: 0.9725 - accuracy: 0.6511
Epoch 00001: val_loss improved from inf to 1.23320, saving model to drive/My Drive/Face Emotion Recognition/Emotion_vgg2.h5
357/357 [==============================] - 69s 189ms/step - loss: 0.9725 - accuracy: 0.6511 - val_loss: 1.2332 - val_accuracy: 0.5495 - lr: 0.0010
Epoch 2/25
357/357 [==============================] - ETA: 0s - loss: 0.9739 - accuracy: 0.6475
Epoch 00002: val_loss improved from 1.23320 to 1.13977, saving model to drive/My Drive/Face Emotion Recognition/Emotion_vgg2.h5
357/357 [==============================] - 68s 190ms/step - loss: 0.9739 - accuracy: 0.6475 - val_loss: 1.1398 - val_accuracy: 0.5628 - lr: 0.0010
Epoch 3/25
357/357 [==============================] - ETA: 0s - loss: 0.9654 - accuracy: 0.6509
Epoch 00003: val_loss did not improve from 1.13977
357/357 [==============================] - 69s 193ms/step - loss: 0.9654 - accuracy: 0.6509 - val_loss: 1.2623 - va